### 필수과제1
- base로만 진행했지만, 추가적으로 임베딩을 다양하게 진행해 보시면서 0.845 의 성능보다 더 올리기
- 다양한 텍스트 전처리를 통해 성능 개선하기
- 파생변수를 추가해도 괜찮음

In [1]:
! pip install nltk

In [25]:
import nltk
from nltk.corpus import movie_reviews, stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import random
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer

# NLTK의 movie_reviews 데이터셋 다운로드
nltk.download('movie_reviews')

# 영화 리뷰와 해당 라벨을 데이터셋으로 로드
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# 데이터를 셔플링하여 무작위로 섞음
random.shuffle(documents)

# 데이터 샘플 확인
sample_review, sample_label = documents[0]
print(f"Review Sample: {' '.join(sample_review[:100])}...")
print(f"Label: {sample_label}")

# 불용어
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Review Sample: hollywood never fails to astound me . every time i think those coked - up little buggers have hit rock bottom ; they come up with a new excavating tool . i was truly convinced that " wild wild west " marked 1999 studio filmmaking at its most hapless , but then along comes " end of days " to prove me wrong . this big budget , brain dead apocalyptic thriller bludgeons its audience with overwrought music , grisly violence , explosions galore and lots of cheesy special effects . the result ? nothing but groans and yawns ....
Label: neg


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
# 리뷰 텍스트와 라벨 분리하고, 불용어 제거
reviews = [' '.join([word for word in review if word.lower() not in stop_words])  for review, category in documents]
label = [category for review, category in documents]

# 리뷰데이터 임베딩 변환
Vactorizer = CountVectorizer()
X = Vactorizer.fit_transform(reviews)
y = label

# 데이터셋 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 111)

# Navie Bayes 분류기 학습
model = MultinomialNB()
model.fit(X_train, y_train)

# 예측값
y_pred = model.predict(X_test)

# 정확도 계산
acc = accuracy_score(y_test, y_pred)
print(f"Base Model Accuracy: {acc: .4f}")

Base Model Accuracy:  0.8275


In [27]:
# NLTK의 movie_reviews 데이터셋 다운로드
nltk.download('movie_reviews')
nltk.download('wordnet')

# 영화 리뷰와 해당 라벨을 데이터셋으로 로드
documents = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

# 데이터를 셔플링하여 무작위로 섞음
random.shuffle(documents)

# 불용어
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
# 1. 문장 및 단어 정규화
def normalize_text(text):
  text = text.lower() # 소문자
  text = re.sub(r'\d+', '', text) # 숫자제거
  text = re.sub(r'[^\w\s]', '', text) # 특수문자 제거
  return text

normalized_reviews = [normalize_text(' '.join(review)) for review, category in documents]

# 2. 어간추출 (stemming)
stemmer = PorterStemmer()
stemmed_reviews = [' '.join([stemmer.stem(word) for word in review.split()]) for review in normalized_reviews]

# 3. 표제어추출
lemmatizer = WordNetLemmatizer()
lemmatized_reviews = [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in stemmed_reviews]

In [ ]:
vectorizer = CountVectorizer()

# 4. CounterVectorizer 하이퍼파라미터 조정
# 최소 빈도수 설정해서 희귀 단어 제거 가능

# 2번의 어간추출, 표제어추출로 학습 테스트
vectorizer_min_df = CountVectorizer(min_df = 2) # min_df=2 → 최소 2개 이상의 문서에서 등장하는 단어만 사용 (희귀 단어 제거)
X_min_df = vectorizer_min_df.fit_transform(lemmatized_reviews)

# N-gram 사용 (2,2) 
vectorizer_bigram = CountVectorizer(ngram_range = (2,2)) # ngram_range=(2,2) → 2개의 단어씩 묶어서 벡터화
X_bigram = vectorizer_bigram.fit_transform(lemmatized_reviews)

# TF-IDF -> 자주 등장하는 일반적인 단어의 중요도를 낮추고, 특정 문서에서만 많이 등장하는 단어의 가중치를 높임
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(lemmatized_reviews)

## 데이터 분할 (어떤 벡터화 방법이 가장 적절한지 확인해보기)

# 1. stem
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(lemmatized_reviews, [label for _, label in documents], test_size = 0.2, random_state = 111)

# 2. min_df 
X_train_min_df, X_test_min_df, y_train_min_df, y_test_min_df = train_test_split(X_min_df, [label for _, label in documents], test_size = 0.2, random_state = 111)

# 3. bigram
X_train_bigram, X_test_bigram, y_train_bigram, y_test_bigram = train_test_split(X_bigram, [label for _, label in documents], test_size = 0.2, random_state = 111)

# 4. TF-IDF
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, [label for _, label in documents], test_size = 0.2, random_state = 111)

## Naive Bayes 분류기 사용하여 학습, 평가
# 텍스트 분류에서 자주 사용되는 베이즈 모델 -> 단어 등장 확률을 기반으로 문서가 특정 클래스에 속할 확률을 계산
# 1. stem
model_stem = MultinomialNB()
model_stem.fit(vectorizer.fit_transform(X_train_stem), y_train_stem)
y_pred_stem = model_stem.predict(vectorizer.transform(X_test_stem))
accuracy_stem = accuracy_score(y_test_stem, y_pred_stem)

# 2. min_df
model_min_df = MultinomialNB()
model_min_df.fit(X_train_min_df, y_train_min_df)
y_pred_min_df = model_min_df.predict(X_test_min_df)
accuracy_min_df = accuracy_score(y_test_min_df, y_pred_min_df)

# 3. bigram
model_bigram = MultinomialNB()
model_bigram.fit(X_train_bigram, y_train_bigram)
y_pred_bigram = model_bigram.predict(X_test_bigram)
accuracy_bigram = accuracy_score(y_test_bigram, y_pred_bigram)

# 4. TF-IDF
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test_tfidf, y_pred_tfidf)

# 각 벡터화 방법별 정확도(accuracy)를 비교
# 가장 높은 정확도를 보이는 방법을 선택하여 최종 모델을 결정 가능

{
    'lemma & CountVectorizer': accuracy_stem,
    'lemma & min_df': accuracy_min_df,
    'lemma & bigram': accuracy_bigram,
    'lemma & tfidf': accuracy_tfidf
}

{'lemma & CountVectorizer': 0.82,
 'lemma & min_df': 0.82,
 'lemma & bigram': 0.8625,
 'lemma & tfidf': 0.8}

### 전처리

In [ ]:
# 정규표현식을 통한 토크나이저 재정의
# 그리드 서치를 통한 하이퍼파라미터 조정 (alpha)

# 이 외에도 다양한 시도를 통해 최종적으로 성능을 높이는 것이 이번 필수과제의 목표

In [35]:
# 그리드서치
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]}
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv = 5, scoring = 'accuracy')
grid_search.fit(X_train_bigram, y_train_bigram)

# 최적의 하이퍼파라미터와 성능 출력
print(f"최적의 파라미터: {grid_search.best_params_}")
print(f"최고의 교차 검증 점수: {grid_search.best_score_}")

최적의 파라미터: {'alpha': 1.0}
최고의 교차 검증 점수: 0.83125
